<a href="https://colab.research.google.com/github/K-Anand-Naik/Compititive-programming/blob/main/Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Momentum 

Momentum-based Trading is based on the assumption that Stocks which have performed well in the past, will perform better in the coming future.
 
To define 'past', we take a variable **N**, and say that : 

<centre> Momentum(For A particular stock) = Close Price(Today) - Close Price(N-day ago) </centre>

This gives us our first hyper-parameter (parameters of the model which could be changed in order to improve the model) : **N**

We would also be reshuffling our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp) at certain intervals of time, which gives us our second hyper-parameter: **T** (The time after which we'll be reshuffling our Portfolio)

Its never suggested to keep all your money invested, you must have some risk-free assets as well, so that even if you lose some of your cash in trading, you could still place better bets and regain that lost cash, Thus, We get our third Hyper-parameter: **R**, The Ratio of Total Balance, which we will using for investing.

You will not be investing in all the 30 Tickers now, Will you? You will choose the top few stocks, which show the highest promise in terms of Momentum, which brings us to another hyper-parameter: **M**, The Number of Top few stocks (based on Momentum), which you'll keep in your Portfolio.

Finally, There's some brokerage fee which you need to pay in order to place orders on the stock market, typically its less than 0.05% of the total amount : **F**


In [4]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

#Declaring the Hyperparameters

N = 50
T = 7
R = 0.8
M = 5
F = 0.0005   # 0.5% Brokerage fee

The Second step would be to define a function which reads the Prices of various Stocks into memory.

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!) 

In [3]:
def GetData(NameOfFile):
  #START CODE HERE
  k=pd.read_csv(NameOfFile)
  k=k[['datadate','tic','adjcp']]
  
  return k  # pd.DataFrame Object

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later. 

In [2]:
def PartitionData(Data):
  DateToIndex=Data.datadate.unique()
  partdata=[]
  k=0

  for i in DateToIndex:
    s=Data.loc[Data['datadate']==DateToIndex[k]]
    s=s.drop(['datadate'],axis=1)
    s=s.to_numpy()
    partdata.append(s)
    k=k+1
  DateToIndex=dict(enumerate(DateToIndex.flatten(), 0))
  
  return partdata, DateToIndex    # List containing of the Data Partitioned according to Date, and the Dictionary mapping Dates to their index in the list


Now, We need a function to calculate the Momentum value for all of our 30 Tickers.
To do this, We need to have a few things in mind:


1.   We need to start at Nth day in our list, as only then we'll be able to calculate the Momentum (This will be taken care of by later parts of the Program, when we actually run the Model)

2.   The Stock Market isn't open on all days, so we often won't be able to go N days behind, and will have to take the closest value instead(We can't just go N entries behind in the List we created and expect that to work, Why?) In order to work with dates, you should get to know more about the datetime library of Python from [here](https://thispointer.com/python-how-to-convert-datetime-object-to-string-using-datetime-strftime/) (Especially the datetime.strftime() function) and about the [datetime.timedelta()](https://www.studytonight.com/python-howtos/how-to-add-days-to-date-in-python) function.

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!

After we calculate the Momentum for all our Tickers, it would be a great thing to do, if we could divide their prices by their mean(in the N day interval, as we need to see which stock outperforms others and in order to do this, it won't be fair if we take the absolute growth in numbers!(Why?)



In [1]:
def GetMomentumBasedPriority(PartitionedDataFrameList, DateToIndex ,today):
  # PartitionedDataFrameList : Pandas DataFrame, The Output of your last function
  # DateToIndex : Dictionary mapping dates to their index in the PartitionedDataFrameList
  # today :  Today's date (string) In Format: YYYYMMDD


  #NdaysAgo is a datatime.date() object contining the required data, you need to convert it to a string and then check if its
  #actually there in the Data you have or will you have to get going using some other nearest date

  NdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N)

  #START CODE HERE!
  NdaysAgo=str(NdaysAgo)
  NdaysAgo=int(NdaysAgo[0:4]+NdaysAgo[5:7]+NdaysAgo[8:])
  j=0
  closestDays=[0,0,0,0]
  for i in [-2,-1,1,2]:
    closestDays[j] = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N+i)  
    closestDays[j] =str(closestDays[j] )
    closestDays[j] =int(closestDays[j] [0:4]+closestDays[j] [5:7]+closestDays[j] [8:])
    j=j+1
  
  DateToIndex=np.array(list(DateToIndex.items()))[:,1]
  if NdaysAgo in DateToIndex:
    print('N day ago was working')
    prevday=NdaysAgo
  else:
    if closestDays[1] in DateToIndex:
      print('N-1 is working')
      prevday=closestDays[1]
    else:
      if closestDays[2] in DateToIndex:
        print('N+1 is working')
        prevday=closestDays[2]
      else:
        if closestDays[0] in DateToIndex:
          print('N-2 is working')
          prevday=closestDays[0]
        elif closestDays[3] in DateToIndex:
          print('N+2 is working')
          prevday=closestDays[3]
        else:
          print('ERROR: this date lies in 5 or more days of holiday')
          prevday=0
  
  momentum=PartitionedDataFrameList[np.where(DateToIndex==int(today))[0][0]][:,1]-PartitionedDataFrameList[np.where(DateToIndex==int(prevday))[0][0]][:,1]
  meancontent=[]
  for i in range(np.where(DateToIndex==int(prevday))[0][0],int(np.where(DateToIndex==int(today))[0][0])+1):
    meancontent.append(PartitionedDataFrameList[i][:,1])
  meancontent=np.asarray(meancontent)
  means=[]
  for i in range(0,meancontent.shape[1]):
    means.append(np.mean(meancontent[:,i]))
  means=np.asarray(means)
  weights=[]
  weights.append(PartitionedDataFrameList[1][:,0])
  weights.append(momentum/means)
  weights=np.asarray(weights)
  return weights   #Numpy Array containing the Momentum divided by mean(in the N-day period considered) of all the 30 tickers, in the required order.



Even after you have got your Momentum-based priorities, and have decided which stocks to buy and what will be the weight of each, you still need to figure out how much of each will you buy. To do this, first you'll sell all your pre-owned stocks which will increase your cash in hand, then you'll know the stocks to buy and their relative weights (given by their Momentum/mean) and you need a function which tells you how many stocks to buy for each ticker!

In [5]:
def GetBalanced(prices, weights,balance):
  balance=R*balance
  topw=np.sort(weights[1])[::-1][0:M]
  k1=weights[1]
  topwname=[]
  topwnameind=[]
  for i in topw:
    topwnameind.append(int(list(k1).index(i)))
    topwname.append(weights[0][list(k1).index(i)])
  topM=np.array([np.array(topwname),topw])
  sharecount=topM
  const=sum(sharecount[1,:])
  for i in range(0,len(sharecount[0])):
    j=int(np.where(prices[:,0]==sharecount[0,i])[0])
    sharecount[1,i]=(sharecount[1,i]*balance)/(prices[int(np.where(prices[:,0]==sharecount[0,i])[0]),1]*const)
  return np.array([topwnameind,sharecount[1,:]])



  
   

Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

Features : 


1.   Your Initial Balance
2.   Your Current Balance
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)

Functionalities: 



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!) 
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Rebalance Portfolio  (Takes Numpy array as input)
5.   Function to change the value of most recent prices stored (Takes Numpy array as input)





In [7]:
class PortFolio:
  def _init_(self, initbalance, posstnstock, recentprice):
    self.initbalance=initbalance
    self.crbalance=initbalance
    self.posstnstock=np.array(posstnstock)
    self.recentprice=np.array(recentprice)



  def SellStock(self,index):
    saleprice=float(self.posstnstock[index,1])*float(self.recentprice[index,1])
    self.posstnstock[index,1]=0
    saleprice=saleprice*(1-F)
    self.crbalance=self.crbalance+saleprice
    return saleprice, self.posstnstock, self.crbalance
  # index : The index of the Stock to sell (0-29)
  
  def BuyStock(self, index, number):
    self.posstnstock[index,1]=float(self.posstnstock[index,1])+number
    self.crbalance=self.crbalance-(float(self.recentprice[index,1])*number*(1+F))
    return self.posstnstock, self.crbalance
  #index : The index of the Stock to buy (0-29) 
  #number : Number of shares to buy (float)

  def CalculateNetWorth(self):
    sharecost=np.asfarray(self.posstnstock[:,1])*np.asfarray(self.recentprice[:,1])
    networth=sum(sharecost)+self.crbalance
    return networth
  #Return Net Worth (All Shares' costs+ Balance)



  def ChangePricesTo(self, newPriceVector):
    self.recentprice=np.array(newPriceVector)
    return self.recentprice
  # newPriceVector : Numpy array containing the prices of all the stocks for the current day
    



# a,b=PartitionData(GetData("DATA.csv"))
# w=GetMomentumBasedPriority(a,b,'20191113')
# pri=np.array(list(b.items()))[:,1]
# pri=a[int(np.where(pri==int('20191113'))[0])]
# #print(a[np.where(b==int('2019113'))[0]])
# #print('asvm',GetBalanced(pri,w,1000)[1,0])
# posstock=[['AAPL', 100], ['AXP',100.26], ['BA',100.5], ['CAT',100.49], ['CSCO',100.46], ['CVX',100.28], ['DD',100.95], ['DIS',100.72], ['GS',100.32], ['HD',100.8], ['IBM',100.48], ['INTC',100.89], ['JNJ',100.27], ['JPM',100.48], ['KO',100.41], ['MCD',100.0], ['MMM',100.55], ['MRK',100.82], ['MSFT',100.31], ['NKE',100.29], ['PFE',100.6], ['PG',100.65], ['RTX',100.31], ['TRV',100.13], ['UNH',100.57], ['V',100.41], ['VZ',100.41], ['WBA',100.25], ['WMT',100.98], ['XOM',100.8]]
# recentstock=[['AAPL', 264.47], ['AXP',120.26], ['BA',362.5], ['CAT',144.49], ['CSCO',48.46], ['CVX',122.28], ['DD',69.95], ['DIS',148.72], ['GS',219.32], ['HD',234.8], ['IBM',134.48], ['INTC',57.89], ['JNJ',131.27], ['JPM',128.48], ['KO',52.41], ['MCD',195.0], ['MMM',170.55], ['MRK',84.82], ['MSFT',147.31], ['NKE',91.29], ['PFE',36.6], ['PG',120.65], ['RTX',148.31], ['TRV',134.13], ['UNH',253.57], ['V',179.41], ['VZ',59.41], ['WBA',62.25], ['WMT',120.98], ['XOM',68.8]]
# p=PortFolio(100000,posstock,recentstock)
# print(p.CalculateNetWorth())
# p.SellStock(0)
# p.BuyStock(0,10)
# p.CalculateNetWorth()
# recentstock=[['AAPL', 0.47], ['AXP',0.26], ['BA',0.5], ['CAT',144.49], ['CSCO',48.46], ['CVX',122.28], ['DD',69.95], ['DIS',148.72], ['GS',219.32], ['HD',234.8], ['IBM',134.48], ['INTC',57.89], ['JNJ',131.27], ['JPM',128.48], ['KO',52.41], ['MCD',195.0], ['MMM',170.55], ['MRK',84.82], ['MSFT',147.31], ['NKE',91.29], ['PFE',36.6], ['PG',120.65], ['RTX',148.31], ['TRV',134.13], ['UNH',253.57], ['V',179.41], ['VZ',59.41], ['WBA',62.25], ['WMT',120.98], ['XOM',68.8]]
# p.ChangePricesTo(recentstock)
# p.CalculateNetWorth()
# p.RebalancePortFolio(w)

With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [ ]:
myPortfolio = PortFolio(________)
NetWorthAfterEachTrade = [___]


#First Get the Data
Data = ______________
PartitionedData, DateToIndex= _______________

#


#Start processing from the (N+1)th Day(among the ones recorded in the Data)
for i in range(__________,_________):
  # Change the Prices to the ith Term
  # Get NetWorth and store in list
  # Check if you need to rebalance Portfolio's Today
  # If so, do it by Calling first the GetMomentumBasedPriority function and then passing it to the rebalance function





  

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

You should try changing the hyper-parameters to increase(/decrease) your performance!


In [ ]:
def VizualizeData(________________):









  

You may use this cell to write about what results you got!